In [60]:
import pandas as pd
import numpy as np
# # set up dictionaries of parameter options for each learner
param_opts = {
    'DecisionTreeClassifier': {},
    'GradientBoostingClassifier': {},
    'KNeighborsClassifier': {},
    'LogisticRegression': {},
    'RandomForestClassifier': {},
    'SVC': {}
}
ml_p = pd.read_csv('ml_p_options.csv')
ml_p.rename(columns={'alg_name':'classifier'},inplace=True)
for ml, df_ml in ml_p.groupby('classifier'):
    for p, df_ml_p in df_ml.groupby('parameters'):
        d = eval(p)
        for keys,v in d.items():
            if keys not in param_opts[ml].keys():
                param_opts[ml][keys] = [v]
            elif v not in param_opts[ml][keys]:
                param_opts[ml][keys].append(v)
print(param_opts)

{'GradientBoostingClassifier': {'learning_rate': [0.01, 0.1, 1], 'subsample': [0.5, 1], 'min_samples_leaf': [10, 20, 5], 'max_depth': [1, 10, 5], 'min_samples_split': [10, 20, 5], 'n_estimators': [100, 250], 'max_features': ['log2', 'sqrt']}, 'SVC': {'tol': [0.0001, 0.001, 0.01, 0.1, 1e-05], 'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 25], 'kernel': ['poly', 'rbf']}, 'DecisionTreeClassifier': {'min_samples_leaf': [10, 20, 5], 'criterion': ['entropy', 'gini'], 'min_samples_split': [10, 20, 5], 'max_depth': [2, 4, 6, 8]}, 'LogisticRegression': {'dual': ['false', 'true'], 'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 25], 'penalty': ['l1', 'l2']}, 'KNeighborsClassifier': {'weights': ['distance', 'uniform'], 'p': [1, 2], 'n_neighbors': [1, 11, 3, 5, 7, 9]}, 'RandomForestClassifier': {'criterion': ['entropy', 'gini'], 'n_estimators': [100, 250], 'min_samples_leaf': [10, 20, 5], 'min_samples_split': [10, 20, 5], 'bootstrap': ['false', 'true'], 'max_features': ['log2', 'sqrt']}}


In [61]:
import pandas as pd
import numpy as np

data = pd.read_csv('sklearn-benchmark5-data-edited.tsv.gz', sep='\t', names=['dataset',
                                                                     'classifier',
                                                                     'parameters',
                                                                     'accuracy', 
                                                                     'macrof1',
                                                                     'bal_accuracy']).fillna('')
print(data.head())
data['accuracy'] = data['accuracy'].apply(lambda x: round(x, 3))
print('loaded ',data['dataset'].unique().shape[0],'datasets and ', data['classifier'].unique().shape[0],'classifiers')
# subset data to classifiers used in PennAI
pennai_classifiers = ['LogisticRegression', 'RandomForestClassifier', 'SVC', 
                      'KNeighborsClassifier', 'DecisionTreeClassifier', 'GradientBoostingClassifier']
mask = np.array([c in pennai_classifiers for c in data['classifier'].values])
data = data.loc[mask,:]
print('datasets (',len(data['dataset'].unique()),')')
print('classifiers (',len(data['classifier'].unique()),'):',data['classifier'].unique())
for ml, df_g in data.groupby('classifier'):
    print('parameters for ',ml,'(',len(df_g['parameters'].unique()),'):',df_g['parameters'].unique()[:5])


/home/bill/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                                             dataset          classifier  \
0  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...  AdaBoostClassifier   
1  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...  AdaBoostClassifier   
2  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...  AdaBoostClassifier   
3  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...  AdaBoostClassifier   
4  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...  AdaBoostClassifier   

                             parameters  accuracy   macrof1 bal_accuracy  
0    learning_rate=0.01,n_estimators=10  0.479375  0.470857   0.479375.1  
1   learning_rate=0.01,n_estimators=100  0.476875  0.476497     0.476875  
2  learning_rate=0.01,n_estimators=1000  0.488125  0.488115     0.488125  
3    learning_rate=0.01,n_estimators=50  0.483750  0.483652      0.48375  
4   learning_rate=0.01,n_estimators=500  0.495625  0.495609     0.495625  
loaded  166 datasets and  13 classifiers
datasets ( 166 )
classifiers ( 6 ): ['DecisionTreeCl

In [62]:
from collections import OrderedDict

def convert_params(params):
    """convert from sklearn-style parameter formatting to dictionary, PennAI style."""
    pdict = {}
    for kv in params.split(','):
        if len(kv)==0: continue
#         print(kv.split('='))
        pdict[str(kv.split('=')[0])] = kv.split('=')[1]
    for k,v in pdict.items():
        try:
            pdict[k] = int(v)
        except ValueError:
            try:     
                pdict[k] = float(v)
            except ValueError:
                pass

    return OrderedDict(sorted(pdict.items()))


data['parameters'] = data['parameters'].apply(lambda x: convert_params(x))

In [63]:
for ml, df_g in data.groupby('classifier'):
    print('parameters for ',ml,'(',len(df_g['parameters'].apply(str).unique()),'):',
           df_g['parameters'].apply(str).unique()[0])

parameters for  DecisionTreeClassifier ( 154 ): OrderedDict([('criterion', 'entropy'), ('max_features', 0.1), ('min_weight_fraction_leaf', 0.0)])
parameters for  GradientBoostingClassifier ( 6301 ): OrderedDict([('learning_rate', 0.01), ('loss', 'deviance'), ('max_depth', 1), ('max_features', 0.2), ('n_estimators', 10)])
parameters for  KNeighborsClassifier ( 54 ): OrderedDict([('n_neighbors', 1), ('weights', 'distance')])
parameters for  LogisticRegression ( 240 ): OrderedDict([('C', 0.5), ('dual', 'False'), ('fit_intercept', 'False'), ('penalty', 'l1')])
parameters for  RandomForestClassifier ( 770 ): OrderedDict([('criterion', 'entropy'), ('max_features', 0.1), ('min_weight_fraction_leaf', 0.0), ('n_estimators', 10)])
parameters for  SVC ( 1239 ): OrderedDict([('C', 0.01), ('coef0', 0.0), ('degree', 2), ('gamma', 0.01), ('kernel', 'poly')])


In [64]:
def valid_param_combo(ml,params):
#     print('ml',ml,type(ml).__name__)
#     print('params',params,type(params).__name__)

    for k,v in params.items():
        if k in param_opts[ml].keys():
            if v not in param_opts[ml][k]:
#                 print('eliminating',params,'for',ml,'with',k,':',v)
                return False
    return True

mask = [valid_param_combo(row['classifier'],row['parameters']) for _, row in data.iterrows()]
    
data_filtered = data.loc[mask]

# data_filtered = data.loc[lambda x: valid_param_combo(i['classifier'],i['parameters']) for i in x]


In [20]:
print(data[data['classifier']=='KNeighborsClassifier']['parameters'].apply(str).unique())

["{'criterion': 'gini', 'learning_rate': 50, 'kernel': 'sigmoid', 'weights': 'uniform', 'penalty': 'l2', 'loss': 'deviance', 'gamma': 50.0, 'degree': 2, 'min_weight_fraction_leaf': 0, 'coef0': 50.0, 'dual': 'True', 'fit_intercept': 'True', 'max_depth': 'None', 'max_features': 'sqrt', 'C': 0.01, 'n_estimators': 500, 'n_neighbors': 9}"]


In [44]:
data_filtered.head()

,dataset,classifier,parameters,accuracy,macrof1,bal_accuracy
175,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'criterion': 'entropy', 'max_features': 0.1, ...",0.507,0.507488,0.5075
176,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'criterion': 'gini', 'max_features': 0.1, 'mi...",0.476,0.476040,0.47625
177,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'criterion': 'entropy', 'max_features': 0.25,...",0.507,0.506832,0.506875
178,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'criterion': 'gini', 'max_features': 0.25, 'm...",0.490,0.489993,0.49
179,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'criterion': 'entropy', 'max_features': 0.5, ...",0.512,0.512497,0.5125


In [65]:
print('data_filtered shape: ', data_filtered.shape)
print('datasets (',len(data_filtered['dataset'].unique()),')')
print('classifiers (',len(data_filtered['classifier'].unique()),'):',data_filtered['classifier'].unique())
for ml, df_g in data_filtered.groupby('classifier'):
    print('parameters for ',ml,'(',len(df_g['parameters'].apply(str).unique()),'):',
           df_g['parameters'].apply(str).unique()[:5])

data_filtered shape:  (113740, 6)
datasets ( 165 )
classifiers ( 5 ): ['DecisionTreeClassifier' 'GradientBoostingClassifier'
 'KNeighborsClassifier' 'RandomForestClassifier' 'SVC']
parameters for  DecisionTreeClassifier ( 154 ): ["OrderedDict([('criterion', 'entropy'), ('max_features', 0.1), ('min_weight_fraction_leaf', 0.0)])"
 "OrderedDict([('criterion', 'gini'), ('max_features', 0.1), ('min_weight_fraction_leaf', 0.0)])"
 "OrderedDict([('criterion', 'entropy'), ('max_features', 0.25), ('min_weight_fraction_leaf', 0.0)])"
 "OrderedDict([('criterion', 'gini'), ('max_features', 0.25), ('min_weight_fraction_leaf', 0.0)])"
 "OrderedDict([('criterion', 'entropy'), ('max_features', 0.5), ('min_weight_fraction_leaf', 0.0)])"]
parameters for  GradientBoostingClassifier ( 36 ): ["OrderedDict([('learning_rate', 0.01), ('loss', 'deviance'), ('max_depth', 1), ('max_features', 'log2'), ('n_estimators', 100)])"
 "OrderedDict([('learning_rate', 0.01), ('loss', 'deviance'), ('max_depth', 1), ('max_f

# remove big datasets

In [66]:
#datasets to remove:
big_datasets = ['poker', 'kddcup', 'sleep', 'fars', 'mnist', 'connect-4', 'shuttle', 'adult', 'krkopt', 
                'letter', 'magic', 'nursery', 'pendigits', 'coil2000', 'agaricus-lepiota','optdigits']
mask = np.array([d not in big_datasets for d in data_filtered['dataset'].values])
clean_data = data_filtered.loc[mask,:]
clean_data.groupby('dataset').count()
print(len(clean_data['dataset'].unique()),'datasets left')

150 datasets left


# remove alg-params that don't cover all datasets

In [70]:
clean_data.loc[:, 'algorithm-parameters'] = (                                             
                                       clean_data['classifier'].values + '|' +                     
                                       clean_data['parameters'].apply(str).values) 
all_datasets = np.unique(clean_data['dataset'].values)
algp_toremove = []
for algp, group in clean_data.groupby('algorithm-parameters'):
    if (group['dataset'].count()<150):
        #print(algp, '\n\t is missing results for', [d for d in all_datasets if d not in np.unique(group['dataset'])])
        #print('removing',algp)
        algp_toremove.append(algp)
        
mask = np.array([ap not in algp_toremove for ap in clean_data['algorithm-parameters'].values])
print('removing',np.sum(~mask),'algorithm-parameter combinations')
clean_data = clean_data.loc[mask,:]
print('new size:',len(clean_data))
clean_data.drop('algorithm-parameters',axis=1,inplace=True)

clean_data.groupby('classifier').count()

removing 0 algorithm-parameter combinations
new size: 35850


,dataset,parameters,accuracy,macrof1,bal_accuracy
classifier,,,,,
DecisionTreeClassifier,23100,23100,23100,23100,23100
GradientBoostingClassifier,2700,2700,2700,2700,2700
KNeighborsClassifier,1800,1800,1800,1800,1800
RandomForestClassifier,6600,6600,6600,6600,6600
SVC,1650,1650,1650,1650,1650


# write modified data to file

In [71]:
clean_data.to_csv('sklearn-benchmark5-data-mock_experiment.tsv.gz',compression='gzip',index=False,sep='\t',
                  header=['dataset',
                                 'algorithm',
                                 'parameters',
                                 'accuracy', 
                                 'macrof1',
                                 'bal_accuracy'])